In [1]:
import numpy as np

def pack_decode(H):
    N = H.shape[1]
    packed = np.zeros((N, N + 2), dtype=np.complex64)
    F = np.zeros((N, N), dtype=np.complex64)
    G = np.zeros((N, N), dtype=np.complex64)
    for k in range(N):
        k_sym = (N-k) % N
        H_conj = np.conj(H[:,k_sym])
        F[:,k] = (H[:,k] + H_conj) / 2
        G[:,k] = (H[:,k] - H_conj) / (2j)
    
    packed[:,0] = F[:,0].real + 0j
    packed[:,1:N//2] = F[:,1:N//2]
    packed[:,N//2] = F[:,N//2].real + 0j

    packed[:,N//2+1:N] = G[:,N//2+1:N]
    packed[:,N] = G[:,0].real + 0j
    packed[:,N+1] = G[:,N//2].real + 0j
    
    return packed

def unpack_encode(packed):
    N = packed.shape[1] - 2
    H = np.zeros((N, N), dtype=np.complex64)
    F = np.zeros((N, N), dtype=np.complex64)
    G = np.zeros((N, N), dtype=np.complex64)
    
    F[:,0] = packed[:,0].real
    F[:,1:N//2] = packed[:,1:N//2]
    F[:,N//2] = packed[:,N//2].real
    
    G[:,N//2+1:N] = packed[:,N//2+1:N]
    G[:,0] = packed[:,N].real
    G[:,N//2] = packed[:,N+1].real
    
    for k in range(1, N//2):
        F[:,N - k] = np.conj(F[:,k])
        G[:,k] = np.conj(G[:,N - k])
    
    H = F + 1j * G
    return H

N = 8
np.random.seed(42)

f = np.random.randn(N,N).astype(np.float32)
g = np.random.randn(N,N).astype(np.float32)
h = f + 1j * g

u = np.random.randn(N,N).astype(np.float32)
v = np.random.randn(N,N).astype(np.float32)
w = u + 1j*v

def fft(h0):
    H_1 = np.fft.fft(h0,axis=1)
    H_1 = pack_decode(H_1)
    H = np.fft.fft(H_1,axis=0)
    return H

def ifft(H):
    H_1 = np.fft.ifft(H,axis=0)
    H_1 = unpack_encode(H_1)
    h0 = np.fft.ifft(H_1,axis=1)
    return h0

h1 = ifft(fft(h) * fft(w))

h2 = np.fft.ifft2(np.fft.fft2(f) * np.fft.fft2(u)) + \
     np.fft.ifft2(np.fft.fft2(g) * np.fft.fft2(v)) * 1j

print(np.linalg.norm(h1-h2))



3.991170802742183e-06
